In [3]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from datetime import datetime
from dcon import get_data
from mail import send_mail
import time
import os
import glob
# import holiday file from E\Config
import sys
sys.path.append(r'E:\_Config')  # Assuming 'holiday.py' is in this directory
from holiday import holiday


# check if today is friday or holiday. if holiday or friday then exit the program
def is_today_holiday(holiday_list):
    today_date = datetime.now().date().strftime('%Y-%m-%d')
    return today_date in holiday_list

def is_today_friday():
    return datetime.now().weekday() == 4  # Friday is 4

if is_today_holiday(holiday()):
    print("Today is a holiday. Exiting program.")
    sys.exit()

if is_today_friday():
    print("Today is Friday. Exiting program.")
    sys.exit()



zid = 100001
# district = ('Sylhet Retail', 'Rahima enterprise', 'District', 'Chittagong retail', 'Imamgonj') 
salesman_info_dict = {
    'SA--000068': ('mssalim5904@gmail.com', 'Md. Salimullah (Salim)'),
    'SA--000224': ('aponmdarifulislam603@gmail.com', 'Md. Ariful Islam-2'),
    'SA--000038': ('hbelal706@gmail.com', 'Md. Belal Hossain'),
    'SA--000144': ('hossainmobarok45@gmail.com', 'Syed Mobarok Hossain'),
    'SA--000021': ('mdm10413@gmail.com', 'Md. Maruful Islam'),
    'SA--000193': ('limonmridha30@gmail.com', 'Limon Mridha'),
    'SA--000114': ('plabonpavel494@gmail.com', 'Md. Pavel Mia'),
    'SA--000011': ('jamal.hmbr@gmail.com', 'Jamal Hossain Titu'),
    'SA--000192': ('sajibhossen701@gmail.com', 'Sojib Hossen'),
    'SA--000098': ('javedfeni127@gmail.com', 'Md. Belayet Hossen'),
    'SA--000227': ('mdsumonhossionmithu00@gmail.com', 'Md. Sumon Hossain Mithu'),
    'SA--000242': ('iforhadul08@gmail.com', 'Md. Forhadul Islam')
}

salesman = tuple(salesman_info_dict.keys())

# Get Today Date
now_date = datetime.today()
today_date = now_date.strftime("%Y-%m-%d")
# today_date = '2023-12-05'
print (today_date)




2024-01-22


In [5]:
twenty_days_ago = now_date - timedelta (20)
twenty_days_ago = twenty_days_ago.strftime("%Y-%m-%d")


'2024-01-02'

In [10]:
#%%
to_salesman_query = f"""
                        SELECT
                        opdor.xdornum AS do_number,
                        opdor.xcus,
                        CONCAT(opdor.xcus, '--', cacus.xshort) AS customer,
                        cacus.xadd1 AS address,
                        cacus.xtaxnum AS mobile_number,
                        cacus.xcity AS city,
                        opdor.xsp,
                        prmst.xname AS SName,
                        opdor.xtotamt AS total_DO_amt,
                        opdor.xdate AS goods_receive_date
                        FROM
                        opdor
                        LEFT JOIN cacus ON opdor.xcus = cacus.xcus AND opdor.zid = cacus.zid
                        JOIN prmst ON opdor.xsp = prmst.xemp
                        WHERE
                        opdor.zid = {zid}
                        AND cacus.zid = {zid}
                        AND prmst.zid = {zid}  -- Corrected the logical error here
                        AND opdor.xdate >= '{twenty_days_ago}'
        """

df_send_to_salesman = get_data (to_salesman_query , fetchOneOrAll = 'all' , df = True, conn = "local")

df_send_to_salesman

,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date
0,DO--460097,CUS-006351,CUS-006351--M/S Al-Macca Trading,"Mahisha Complex Jail Road, Sha Amanat (Rah) Ma...",8801711134514,Chittagong,SA--000144,Syed Mobarok Hossain,22000.00,2024-01-08
1,DO--459616,CUS-004823,CUS-004823--M/S Mahfuj Enterprise,Marwari Potti Joypur Hat,8801770326678,Jaypurhat,SA--000098,Md. Belayet Hossen,24686.00,2024-01-02
2,DO--459746,CUS-003624,CUS-003624--Al Amin Electric,"Jalal Sarkar Market Road, Board Bazar",8801727242928,Board Bazer,SA--000251,Md. Liton Miah-2,1699.00,2024-01-03
3,DO--459694,CUS-000582,CUS-000582--Hamim Sanitary,Shajadpur,,Vatara,SA--000006,Muhammad Zakir Hossain,2300.00,2024-01-02
4,DO--460103,CUS-009964,CUS-009964--Mirab Fancy,"New Market , Kastghor, Mohjon Potti",8801710462542,Sylhet,SA--000021,Md. Maruful Islam,5580.00,2024-01-08
...,...,...,...,...,...,...,...,...,...,...
2078,DO--461683,CUS-001022,CUS-001022--Hero Sanitary,"11 No Vaban Market, Gulshan-2",8801921454015,Gulshan-1,SA--000280,Md. Romon Khan,4050.00,2024-01-21
2079,DO--461687,CUS-001013,CUS-001013--Jakir Hardware Store,"88, D.C.C. Market (South Khacha Bazar)",8801715624445,Gulshan-1,SA--000280,Md. Romon Khan,12800.00,2024-01-21
2080,DO--461685,CUS-004461,CUS-004461--Faruk Hardware,Gulshan-2,,Gulshan-1,SA--000280,Md. Romon Khan,1680.00,2024-01-21
2081,DO--461692,CUS-001032,CUS-001032--Banani Hardware,"47/1, Banani Bazar, (Fayez Mantion) Dhaka-1213.",8801819493313,Banani,SA--000280,Md. Romon Khan,4960.00,2024-01-21


In [16]:
df_send_to_salesman['goods_receive_date'] = pd.to_datetime(df_send_to_salesman['goods_receive_date']) + pd.Timedelta(days=3)
df_send_to_salesman



,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date,date_diff
1777,DO--461373,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,22080.00,2024-01-21,4 days
1557,DO--461161,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,33131.00,2024-01-19,6 days
550,DO--460171,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,261630.00,2024-01-11,14 days
128,DO--459713,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,7920.00,2024-01-06,19 days
35,DO--459610,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,11149.00,2024-01-05,20 days
...,...,...,...,...,...,...,...,...,...,...,...
593,DO--460246,CUS-009988,CUS-009988--M/S Mayer Dowa Sanitary Mart,"1/1 Kalachadpur, Uttar Baridhara, Hotel Bon Vi...",8801786047779,Kalachadpur,SA--000258,Md. Suruj Mia,13500.00,2024-01-11,14 days
643,DO--460290,CUS-009992,CUS-009992--Songita Hardware,"Saver Nama Bazer, Saver",8801922840682,Saver,SA--000255,Sumon Miah,4140.00,2024-01-11,14 days
1500,DO--461106,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000070,Md. Sumon Sheikh,41040.00,2024-01-18,7 days
596,DO--460252,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000108,Md. Anichur Rahman,6060.00,2024-01-11,14 days


In [17]:

df_send_to_salesman['today_date'] = today_date
df_send_to_salesman['date_diff'] = pd.to_datetime(df_send_to_salesman['today_date']) - pd.to_datetime (df_send_to_salesman['goods_receive_date'])
df_send_to_salesman = df_send_to_salesman.sort_values(by=['customer', 'date_diff'], ascending = False)

# this line will need if criteria given
df_send_to_salesman = df_send_to_salesman[df_send_to_salesman['date_diff'].dt.days <= 20].sort_values(by=['customer', 'date_diff']).drop(columns=['today_date'])
df_send_to_salesman


,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date,date_diff
1777,DO--461373,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,22080.00,2024-01-21,1 days
1557,DO--461161,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,33131.00,2024-01-19,3 days
550,DO--460171,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,261630.00,2024-01-11,11 days
128,DO--459713,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,7920.00,2024-01-06,16 days
35,DO--459610,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,11149.00,2024-01-05,17 days
...,...,...,...,...,...,...,...,...,...,...,...
593,DO--460246,CUS-009988,CUS-009988--M/S Mayer Dowa Sanitary Mart,"1/1 Kalachadpur, Uttar Baridhara, Hotel Bon Vi...",8801786047779,Kalachadpur,SA--000258,Md. Suruj Mia,13500.00,2024-01-11,11 days
643,DO--460290,CUS-009992,CUS-009992--Songita Hardware,"Saver Nama Bazer, Saver",8801922840682,Saver,SA--000255,Sumon Miah,4140.00,2024-01-11,11 days
1500,DO--461106,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000070,Md. Sumon Sheikh,41040.00,2024-01-18,4 days
596,DO--460252,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000108,Md. Anichur Rahman,6060.00,2024-01-11,11 days


In [18]:
get_customer_id = tuple(df_send_to_salesman['xcus'].to_list())


# now get their untill previous days balance
today_balance_col = f"till_{today_date}_balance".replace("-","_")

balance_query = f"""
    SELECT gldetail.xsub AS xcus, SUM(gldetail.xprime) AS balance_till_today
    FROM glheader
    JOIN gldetail ON glheader.xvoucher = gldetail.xvoucher
    JOIN cacus ON gldetail.xsub = cacus.xcus
    WHERE glheader.zid = {zid}
        AND gldetail.zid = {zid}
        AND cacus.zid = {zid}
        AND gldetail.xproj = 'GULSHAN TRADING'
        AND gldetail.xvoucher NOT LIKE '%%OB%%'
        AND glheader.xdate <= '{today_date}'
        AND gldetail.xsub IN {get_customer_id}
    GROUP BY gldetail.xsub
    
"""


# Execute the query using psycopg2 not pd.read sql
df_prev_balance_customer = get_data (balance_query , fetchOneOrAll = 'all' , df = True, conn = "local")
df_prev_balance_customer

,xcus,balance_till_today
0,CUS-003711,12999.20
1,CUS-004383,9000.00
2,CUS-005628,13380.00
3,CUS-009141,-1.38
4,CUS-000033,0.00
...,...,...
1502,CUS-003126,0.00
1503,CUS-003497,0.00
1504,CUS-007599,13560.00
1505,CUS-007865,-0.33


In [19]:
# now merge with df_send_to_salesman
df_send_to_salesman_with_balance = pd.merge(df_send_to_salesman, df_prev_balance_customer, on = 'xcus', how='left')
df_send_to_salesman_with_balance

# # for send to salesman
# df_send_to_salesman = df_send_to_salesman.drop(columns=['xcus'])
df_send_to_salesman_with_balance
# filter out which customer balance is less than 100
df_send_to_salesman_with_balance= df_send_to_salesman_with_balance[df_send_to_salesman_with_balance['balance_till_today'] > 300]
df_send_to_salesman_with_balance

,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date,date_diff,balance_till_today
0,DO--461373,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,22080.00,2024-01-21,1 days,1032870.48
1,DO--461161,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,33131.00,2024-01-19,3 days,1032870.48
2,DO--460171,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,261630.00,2024-01-11,11 days,1032870.48
3,DO--459713,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,7920.00,2024-01-06,16 days,1032870.48
4,DO--459610,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,11149.00,2024-01-05,17 days,1032870.48
...,...,...,...,...,...,...,...,...,...,...,...,...
2075,DO--460103,CUS-009964,CUS-009964--Mirab Fancy,"New Market , Kastghor, Mohjon Potti",8801710462542,Sylhet,SA--000021,Md. Maruful Islam,5580.00,2024-01-11,11 days,68579.00
2076,DO--460488,CUS-009966,CUS-009966--M/S Pikash Enterprise,"Malopara, kadirgonj Road, Rajshahi",8801757173165,Rajshahi,SA--000193,Limon Mridha,16440.00,2024-01-13,9 days,16440.00
2080,DO--461106,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000070,Md. Sumon Sheikh,41040.00,2024-01-18,4 days,11459.72
2081,DO--460252,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000108,Md. Anichur Rahman,6060.00,2024-01-11,11 days,11459.72


In [20]:

# get last payment date
last_payment_query = f"""SELECT glheader.xdate as last_payment_date,gldetail.xsub AS xcus, sum(gldetail.xprime) as last_payment
                        FROM glheader
                        JOIN gldetail ON glheader.xvoucher = gldetail.xvoucher
                        JOIN cacus ON gldetail.xsub = cacus.xcus
                        WHERE glheader.zid = {zid}
                            AND gldetail.zid = {zid}
                            AND cacus.zid = {zid}
                            AND gldetail.xproj = 'GULSHAN TRADING'
                            AND gldetail.xvoucher LIKE '%%RCT-%%'
                            AND glheader.xdate <= '{today_date}'
                            AND gldetail.xsub in {get_customer_id}
                            group by  glheader.xdate,gldetail.xsub
                    order by gldetail.xsub, glheader.xdate desc
                    """
df_last_payment = get_data (last_payment_query , fetchOneOrAll = 'all' , df = True, conn = "local")
df_last_payment = df_last_payment.drop_duplicates(subset=['xcus'], keep= 'first' )

In [21]:
df_last_payment['last_payment'] = abs (df_last_payment['last_payment'])
df_last_payment

,last_payment_date,xcus,last_payment
0,2024-01-20,CUS-000001,32440.00
129,2024-01-17,CUS-000003,400000.00
563,2024-01-18,CUS-000015,7570.00
938,2024-01-11,CUS-000020,1050.00
950,2024-01-15,CUS-000022,303.00
...,...,...,...
138706,2023-03-27,CUS-009966,34800.00
138717,2024-01-13,CUS-009977,3950.00
138723,2024-01-09,CUS-009988,13500.00
138769,2024-01-09,CUS-009992,4140.00


In [22]:
# create last payment date column
df_send_to_salesman_with_balance = pd.merge(df_send_to_salesman_with_balance, df_last_payment, on = 'xcus', how='left')
df_send_to_salesman_with_balance


,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date,date_diff,balance_till_today,last_payment_date,last_payment
0,DO--461373,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,22080.00,2024-01-21,1 days,1032870.48,2024-01-20,32440.00
1,DO--461161,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,33131.00,2024-01-19,3 days,1032870.48,2024-01-20,32440.00
2,DO--460171,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,261630.00,2024-01-11,11 days,1032870.48,2024-01-20,32440.00
3,DO--459713,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,7920.00,2024-01-06,16 days,1032870.48,2024-01-20,32440.00
4,DO--459610,CUS-000001,CUS-000001--HMBR FiX iT,"RM Centre-1, Gulshan",,Gulshan-1,SA--000901,Fixit,11149.00,2024-01-05,17 days,1032870.48,2024-01-20,32440.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,DO--460103,CUS-009964,CUS-009964--Mirab Fancy,"New Market , Kastghor, Mohjon Potti",8801710462542,Sylhet,SA--000021,Md. Maruful Islam,5580.00,2024-01-11,11 days,68579.00,2023-12-28,12455.00
549,DO--460488,CUS-009966,CUS-009966--M/S Pikash Enterprise,"Malopara, kadirgonj Road, Rajshahi",8801757173165,Rajshahi,SA--000193,Limon Mridha,16440.00,2024-01-13,9 days,16440.00,2023-03-27,34800.00
550,DO--461106,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000070,Md. Sumon Sheikh,41040.00,2024-01-18,4 days,11459.72,2024-01-11,3000.00
551,DO--460252,CUS-009996,CUS-009996--Goldmine Hardware & Electric,"Shop-1 Under Ground , Hazi Islam Plaza",8801608700313,Damra,SA--000108,Md. Anichur Rahman,6060.00,2024-01-11,11 days,11459.72,2024-01-11,3000.00


In [23]:
df_send_to_salesman_with_balance = df_send_to_salesman_with_balance.sort_values(by = ['xsp','xcus','date_diff'])
df_send_to_salesman_with_balance

,do_number,xcus,customer,address,mobile_number,city,xsp,sname,total_do_amt,goods_receive_date,date_diff,balance_till_today,last_payment_date,last_payment
17,DO--460062,CUS-000656,CUS-000656--Maa Enterprise,"1047, Kanis Plaza, Khilbarirtak, Shahjadpur, V...",8801823232760,Vatara,SA--000006,Muhammad Zakir Hossain,6360.00,2024-01-09,13 days,20000.00,2024-01-14,6360.00
18,DO--460043,CUS-000656,CUS-000656--Maa Enterprise,"1047, Kanis Plaza, Khilbarirtak, Shahjadpur, V...",8801823232760,Vatara,SA--000006,Muhammad Zakir Hossain,28000.00,2024-01-09,13 days,20000.00,2024-01-14,6360.00
23,DO--461549,CUS-000799,CUS-000799--Sonia Supply,Sajahanpur Rale Gate,8801817677318,Malibag,SA--000006,Muhammad Zakir Hossain,3120.00,2024-01-23,-1 days,3119.36,2024-01-06,5300.00
24,DO--459854,CUS-000799,CUS-000799--Sonia Supply,Sajahanpur Rale Gate,8801817677318,Malibag,SA--000006,Muhammad Zakir Hossain,12500.00,2024-01-07,15 days,3119.36,2024-01-06,5300.00
25,DO--461550,CUS-000808,CUS-000808--Rahaman Hardware,"Collage Road, Khilgaon",8801819418349,Malibag,SA--000006,Muhammad Zakir Hossain,3085.00,2024-01-23,-1 days,3084.67,2023-12-23,1465.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,DO--460464,CUS-004199,CUS-004199--General,Vatara More,,Vatara,SA--000902,Personal Sales,870.00,2024-01-13,9 days,92046.80,2024-01-20,760.00
344,DO--460311,CUS-004199,CUS-004199--General,Vatara More,,Vatara,SA--000902,Personal Sales,980.00,2024-01-12,10 days,92046.80,2024-01-20,760.00
339,DO--461261,CUS-004199,CUS-004199--General,Vatara More,,Vatara,SA--000903,Internal Sales,160.00,2024-01-20,2 days,92046.80,2024-01-20,760.00
5,DO--460323,CUS-000003,CUS-000003--Rahima Enterprise,"Shasangasa, Comilla",,Comilla,SA--100000,Rahima Enterprise,128970.00,2024-01-12,10 days,3302608.17,2024-01-17,400000.00


In [24]:
# remove unnecessary column
df_send_to_salesman_with_balance = df_send_to_salesman_with_balance.iloc[:, [0,6,7,2,8,11,9,10,12,13,5]]
df_send_to_salesman_with_balance

,do_number,xsp,sname,customer,total_do_amt,balance_till_today,goods_receive_date,date_diff,last_payment_date,last_payment,city
17,DO--460062,SA--000006,Muhammad Zakir Hossain,CUS-000656--Maa Enterprise,6360.00,20000.00,2024-01-09,13 days,2024-01-14,6360.00,Vatara
18,DO--460043,SA--000006,Muhammad Zakir Hossain,CUS-000656--Maa Enterprise,28000.00,20000.00,2024-01-09,13 days,2024-01-14,6360.00,Vatara
23,DO--461549,SA--000006,Muhammad Zakir Hossain,CUS-000799--Sonia Supply,3120.00,3119.36,2024-01-23,-1 days,2024-01-06,5300.00,Malibag
24,DO--459854,SA--000006,Muhammad Zakir Hossain,CUS-000799--Sonia Supply,12500.00,3119.36,2024-01-07,15 days,2024-01-06,5300.00,Malibag
25,DO--461550,SA--000006,Muhammad Zakir Hossain,CUS-000808--Rahaman Hardware,3085.00,3084.67,2024-01-23,-1 days,2023-12-23,1465.00,Malibag
...,...,...,...,...,...,...,...,...,...,...,...
343,DO--460464,SA--000902,Personal Sales,CUS-004199--General,870.00,92046.80,2024-01-13,9 days,2024-01-20,760.00,Vatara
344,DO--460311,SA--000902,Personal Sales,CUS-004199--General,980.00,92046.80,2024-01-12,10 days,2024-01-20,760.00,Vatara
339,DO--461261,SA--000903,Internal Sales,CUS-004199--General,160.00,92046.80,2024-01-20,2 days,2024-01-20,760.00,Vatara
5,DO--460323,SA--100000,Rahima Enterprise,CUS-000003--Rahima Enterprise,128970.00,3302608.17,2024-01-12,10 days,2024-01-17,400000.00,Comilla


In [35]:
df_send_to_salesman_with_balance = df_send_to_salesman_with_balance[df_send_to_salesman_with_balance['date_diff'] > pd.Timedelta(days=0)]\
                                    .sort_values(by= ['date_diff', 'balance_till_today'], ascending=False)

df_send_to_salesman_with_balance

,do_number,xsp,sname,customer,total_do_amt,balance_till_today,goods_receive_date,date_diff,last_payment_date,last_payment,city
4,DO--459610,SA--000901,Fixit,CUS-000001--HMBR FiX iT,11149.00,1032870.48,2024-01-05,17 days,2024-01-20,32440.00,Gulshan-1
252,DO--459619,SA--000192,Sojib Hossen,CUS-003338--Bismillah Hardware,7499.79,97314.93,2024-01-05,17 days,2024-01-01,17350.00,Jamalpur
496,DO--459618,SA--000192,Sojib Hossen,CUS-008865--Kalam Hardware,13630.00,53670.00,2024-01-05,17 days,2023-11-06,16120.00,Jamalpur
134,DO--459614,SA--000193,Limon Mridha,CUS-002737--Porsha Hardware,14700.00,47155.15,2024-01-05,17 days,2024-01-21,6000.00,Rajshahi
526,DO--459613,SA--000193,Limon Mridha,CUS-009733--Shuvesca Hardware,26000.00,42747.00,2024-01-05,17 days,2023-03-30,16800.00,Rajshahi
...,...,...,...,...,...,...,...,...,...,...,...
519,DO--461380,SA--000021,Md. Maruful Islam,CUS-009668--M/S Aziz Electric,5035.00,5034.01,2024-01-21,1 days,2023-09-18,2944.00,Sylhet
436,DO--461418,SA--000241,Md. Sahen Sha,CUS-006881--Safa Hardware,10015.00,5015.00,2024-01-21,1 days,2024-01-20,5000.00,Tongi
497,DO--461464,SA--000278,Md. Robiul Islam,CUS-008868--Bhadon Hardware & Paint Store,5160.00,5000.00,2024-01-21,1 days,2024-01-20,5160.00,Saver
457,DO--461399,SA--000251,Md. Liton Miah-2,CUS-007392--Khondokar Enterprise,3153.00,1259.04,2024-01-21,1 days,2024-01-20,2403.00,Board Bazer


In [36]:
df_send_to_salesman_with_balance.to_excel("H_71_2_reminder.xlsx", index=False)

In [37]:

# change color of worksheet base ond date difference condition
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Load the existing workbook
wb = load_workbook("H_71_2_reminder.xlsx")
ws = wb.active

# Define the column index for "date_diff"
date_diff_column_index = 8  # Column J

# Define conditions for coloring cells in the "date_diff" column
for row in range(2, ws.max_row + 1):
    date_diff_cell = ws.cell(row=row, column=date_diff_column_index)

    if date_diff_cell.value > 15:
        date_diff_cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Red
    elif 10 <= date_diff_cell.value <= 15:
        date_diff_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow
    else:
        date_diff_cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green

# Save the modified workbook
wb.save("H_71_2_reminder.xlsx")


In [38]:
# send to salesman

for salesman_id, (email, salesman_name) in salesman_info_dict.items():
    print(salesman_id)

    # Filter the DataFrame for the specific salesman_id
    salesman_df = df_send_to_salesman_with_balance[df_send_to_salesman_with_balance['xsp'] == salesman_id]

    # Check if the DataFrame is not empty
    if not salesman_df.empty:
        # Create a unique Excel file for each salesman
        excel_file_path = f"salesman_{salesman_id}.xlsx"
        salesman_df.to_excel(excel_file_path, index=False)

        # Create the HTML body with the filtered DataFrame
        html_body = [(salesman_df, f" Customer Due balance from last 20 days to {today_date})")]

        # Send email with the Excel file as the attachment
        send_mail(
            subject=f"H_71.2 Customer due Balance details from last month till {today_date}",
            bodyText=f"""
            Dear {salesman_name} <br>
            Plese find herewith the attached file regarding customer due balance from last 20 dayas
             """,
            attachment=[excel_file_path],
            recipient=[email, ],
            html_body=html_body
        )
    else:
        print(f"No data for salesman: {salesman_id}")
# delete all excelfile from current directory


SA--000068
SA--000224
SA--000038
No data for salesman: SA--000038
SA--000144
SA--000021
SA--000193
SA--000114
SA--000011
SA--000192
SA--000098
SA--000227
SA--000242


In [39]:
## %%
# For Riyad vai
excel_file_path = f"H_71_2_reminder.xlsx"

html_body = [(df_send_to_salesman_with_balance, f" Customer cumulative balance from last 20 days")]

send_mail(
    subject="H71.2 Customer cumulative balance to reminder salesman",
    bodyText=f"Todays {today_date} mail sent to all salesman and find the excel sheet of district delivered goods",
    attachment=[excel_file_path],
    recipient=['saleshmbrbd@gmail.com', 'ithmbrbd@gmail.com', 'asaddat87@gmail.com' ],
  
)


In [41]:
def delete_xlsx_files():
    try:
        for file in glob.glob("*.xlsx"):
            os.remove(file)
        print("All .xlsx files deleted successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to delete .xlsx files in the current working directory
delete_xlsx_files()

All .xlsx files deleted successfully.
